# Capstone Project - The Battle of Neighborhoods

## Introduction/Business Problem

Introduction where you discuss the business problem and who would be interested in this project.

#### "Would you recommend a location in Vancouver to open a new Seafood restaurant ?"  


Vancouver is closed to the sea therefore it's quite famous for its seafood. My boss, an American investor just moved here and decided to buy one Seafood restaurant and make it the best seafood restaurants in Vancouver.

He has tried top 3 seafood restaurants elected by OpenTable and rated them. He asked me to investigate the top 4-6 restaurants and sent him a report to help him make decisions. 

He told me his ideal restaurant should 
- be near public transport facilities
- be closed to waterfront


## Data

Data where you describe the data that will be used to solve the problem and the source of the data.

According to the question, I need to find following data to resolve the problem.

#### 1. Stakeholder's choice 

In [1]:
import pandas as pd
df_choice = pd.read_csv('restaurant.csv',index_col=[0])

In [2]:
df_choice

,Name,lat,lon,rating
1,Ancora Ambleside,49.327349,-123.154458,5.0
2,Boulevard Kitchen & Oyster Bar,49.282718,-123.123883,4.0
3,Oddfish Restaurant,49.270894,-123.147585,4.0


#### 2.  Four possible  places  

I randomly chose four places from google map and get the coordinates down into the csv file.

In [3]:
df_cand = pd.read_csv('candidates.csv',index_col=[0])
df_cand

,Address,lat,lon
1,Ferris' Upstairs Seafood and Oyster Bar,48.427011,-123.369044
2,Joe Fortes Seafood & Chop House,49.284791,-123.124556
3,Blue Crab Seafood House,48.421903,-123.379851


#### 3. Nearby venues around places

It is recommended by the stakeholder that the restaurant should be surrounded by some public transport facilities. Therefore, we are going to discover the following three categories. We will use foursquare api to get those info.

In [3]:
fs_categories = {
    'Nightlife Spot': '4d4b7105d754a06376d81259',
    'bus stop': '52f2ab2ebcbc57f1066b8b4f',
    'Shopping Mall':'4bf58dd8d48988d1fd941735'
}

We will use the first stakeholder's choice to testify the function of api.

In [10]:
VERSION = '20180604'
CLIENT_ID = 'FLSHE5WOWUI5JJPQBI1IMJBGK1KL0JX4CXV4DG2ZRQVPICQE'
CLIENT_SECRET = 'PKNCMWET2QX4MY2LIZWBJVNZAB4RRD3T2QBQIJAAWUCPS35C'
latitude = 49.327349
longitude = -123.154458
category = 'Nightlife Spot'
radius = 2000
category_id='4d4b7105d754a06376d81259'

In [11]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&category_id={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,VERSION,category,category_id,radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=FLSHE5WOWUI5JJPQBI1IMJBGK1KL0JX4CXV4DG2ZRQVPICQE&client_secret=PKNCMWET2QX4MY2LIZWBJVNZAB4RRD3T2QBQIJAAWUCPS35C&ll=49.327349,-123.154458&v=20180604&query=Nightlife Spot&category_id=4d4b7105d754a06376d81259&radius=2000'

In [12]:
import requests
result = requests.get(url = url).json()
result

{'meta': {'code': 200, 'requestId': '5e04749478a484001b6fb3cd'},
 'response': {'venues': [{'id': '4abe72c2f964a520ab8d20e3',
    'name': 'White Spot',
    'location': {'address': '797 Main St',
     'lat': 49.32636108016595,
     'lng': -123.13922882080078,
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.32636108016595,
       'lng': -123.13922882080078}],
     'distance': 1110,
     'postalCode': 'V7T 0A5',
     'cc': 'CA',
     'city': 'West Vancouver',
     'state': 'BC',
     'country': 'Canada',
     'formattedAddress': ['797 Main St',
      'West Vancouver BC V7T 0A5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d143941735',
      'name': 'Breakfast Spot',
      'pluralName': 'Breakfast Spots',
      'shortName': 'Breakfast',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1577350317',
    'hasPerk': False},
   {'id': '4f5407cce4b063296ec8ded5',


In [7]:
venues = result['response']['venues']

In [8]:
from pandas.io.json import json_normalize
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4abe72c2f964a520ab8d20e3,797 Main St,CA,West Vancouver,Canada,1110,"[797 Main St, West Vancouver BC V7T 0A5, Canada]","[{'label': 'display', 'lat': 49.32636108016595...",49.326361,-123.139229,V7T 0A5,BC,White Spot,v-1577346481
1,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4f5407cce4b063296ec8ded5,440 Gordon Ave,CA,West Vancouver,Canada,1997,"[440 Gordon Ave, West Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.331499, 'lng':...",49.331499,-123.127669,NaN,BC,Wright-Walger Spot.,v-1577346481


In [9]:
dataframe.name

0             White Spot
1    Wright-Walger Spot.
Name: name, dtype: object

We can see that there are 7 bus stops around the first restaurant in a radius of 500 meters. 
As we still want to discover the nearby venues around the restaurant, we are going to write a function to help us get all those info.

In [155]:
def venues_nearby(lat,lon,category):
    VERSION = '20180604'
    CLIENT_ID = 'FLSHE5WOWUI5JJPQBI1IMJBGK1KL0JX4CXV4DG2ZRQVPICQE'
    CLIENT_SECRET = 'PKNCMWET2QX4MY2LIZWBJVNZAB4RRD3T2QBQIJAAWUCPS35C'
    latitude = lat
    longitude = lon
    category = category
    category_id=fs_categories[category]
    radius = 500
    result = requests.get(url = url).json()
    venues = result['response']['venues']
    dataframe = json_normalize(venues)
    df= dataframe['name']
    return df

Let's test it out. We want to find if there are any waterfront around first restaurant. 

In [144]:
df_choice

,Name,lat,lon,rating
1,Ancora Ambleside,49.327349,-123.154458,5.0
2,Boulevard Kitchen & Oyster Bar,49.282718,-123.123883,4.0
3,Oddfish Restaurant,49.270894,-123.147585,4.0


In [148]:
lat = df_choice.lat[1]
lon = df_choice.lon[1]
lat,lon

(49.327349, -123.154458)

In [165]:
venues_nearby(lat,lon,'Waterfront')

0                    Bus Stop 54602 (250-256,258)
1                Bus Stop 54601 (250/1/2/3/5/6/8)
2    Bus Stop 54600 (250,251,252,253,255,256,258)
3                                  Bus Stop 54780
4                                  Bus Stop 54612
5                                  Bus Stop 54613
6                Winstanley Business Advisors Ltd
Name: name, dtype: object